In [1]:
import numpy as np
import pandas as pd
import anndata as ad
import scanpy as sc
from scipy.sparse import csr_matrix

**Count matrix and metadata for VISp dataset**
 - Download the count data from Allen Institute portal
 - Convert to AnnData format - see [this getting started with AnnData tutorial](https://anndata-tutorials.readthedocs.io/en/latest/getting-started.html)
 - Save the resulting object for later use

In [2]:
# Download count matrices from https://portal.brain-map.org/atlases-and-data/rnaseq/mouse-v1-and-alm-smart-seq or use the shell commands below.
!curl -o ../data/VISp.zip https://celltypes.brain-map.org/api/v2/well_known_file_download/694413985
!unzip -d ../data/VISp ../data/VISp.zip
!rm ../data/VISp.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  291M  100  291M    0     0  54.9M      0  0:00:05  0:00:05 --:--:-- 55.7M
Archive:  ../data/VISp.zip
  inflating: ../data/VISp/mouse_VISp_2018-06-14_exon-matrix.csv  
  inflating: ../data/VISp/mouse_VISp_2018-06-14_genes-rows.csv  
  inflating: ../data/VISp/mouse_VISp_2018-06-14_intron-matrix.csv  
  inflating: ../data/VISp/mouse_VISp_2018-06-14_readme.txt  
  inflating: ../data/VISp/mouse_VISp_2018-06-14_samples-columns.csv  


In [3]:
# Load VISp dataset
filename = '../data/VISp/mouse_VISp_2018-06-14_exon-matrix.csv'
expr_df = pd.read_csv(filename, header=0, index_col=0, delimiter=',').transpose()
expr = expr_df.values

# Find gene names
filename = '../data/VISp/mouse_VISp_2018-06-14_genes-rows.csv'
genes_df = pd.read_csv(filename, header=0, index_col=0, delimiter=',')
gene_symbol = genes_df.index.values
gene_ids = genes_df['gene_entrez_id'].values
gene_names = np.array([gene_symbol[np.where(gene_ids == name)[0][0]] for name in expr_df.columns])

# Get metadata and save restrict to relevant fields
filename = '../data/VISp/mouse_VISp_2018-06-14_samples-columns.csv'
obs = pd.read_csv(filename, header=0, index_col=0, delimiter=',', encoding='iso-8859-1')

obs = obs.reset_index()
obs = obs[['sample_name','seq_name','class','subclass','cluster']]
obs = obs.rename(columns={'sample_name':'sample_id'})
obs = obs.set_index('sample_id')
obs.head()

,seq_name,class,subclass,cluster
sample_id,,,,
F1S4_160108_001_A01,LS-15006_S09_E1-50,GABAergic,Vip,Vip Arhgap36 Hmcn1
F1S4_160108_001_B01,LS-15006_S10_E1-50,GABAergic,Lamp5,Lamp5 Lsp1
F1S4_160108_001_C01,LS-15006_S11_E1-50,GABAergic,Lamp5,Lamp5 Lsp1
F1S4_160108_001_D01,LS-15006_S12_E1-50,GABAergic,Vip,Vip Crispld2 Htr2c
F1S4_160108_001_E01,LS-15006_S13_E1-50,GABAergic,Lamp5,Lamp5 Plch2 Dock5


In [4]:
# compose and store anndata object for efficient read/write
adata = ad.AnnData(X=csr_matrix(expr), dtype=expr.dtype)
adata.var_names = gene_names
adata.var.index.set_names('genes', inplace=True)
adata.obs = obs
adata.write('../data/VISp.h5ad')

/home/rohan/miniconda3/envs/persist-lightning/lib/python3.11/site-packages/anndata/_core/anndata.py:430: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(


**Filtering samples**

The next code block is optional, and requires `VISp_PERSIST_metadata.csv` which contains:
- cell type labels at different resolutions of the taxonomy (see manuscript for details)
- sample ids to filter out non-neuronal cells

In the following, we will
1. restrict cells only to those samples specified in `VISp_PERSIST_metadata.csv`
2. append metadata from `VISp_PERSIST_metadata.csv` to the AnnData object
3. normalize counts, determine highly variable genes using scanpy functions
3. save a filtered AnnData object into a .h5ad file for subsequent use

In [5]:
adata = ad.read_h5ad('../data/VISp.h5ad')
persist_df = pd.read_csv('../data/VISp_PERSIST_metadata.csv')
print(persist_df.head(2))
print(f'\nold shape: {adata.shape}')

adata = adata[adata.obs['seq_name'].isin(persist_df['seq_name']), :]
obs = adata.obs.copy().reset_index()
obs = obs.merge(right=persist_df, how='left', left_on='seq_name', right_on='seq_name')
obs = obs.set_index('sample_id')

adata = ad.AnnData(X=adata.X, dtype=adata.X.dtype, obs=obs, var=adata.var)
print(f'new shape: {adata.shape}')

             seq_name       cell_types_98 cell_types_50 cell_types_25
0  LS-15006_S09_E1-50  Vip Arhgap36 Hmcn1           n70           n66
1  LS-15006_S10_E1-50          Lamp5 Lsp1    Lamp5 Lsp1           n78

old shape: (15413, 45768)
new shape: (13349, 45768)


/home/rohan/miniconda3/envs/persist-lightning/lib/python3.11/site-packages/anndata/_core/anndata.py:430: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(


**Normalization and preliminary gene selection**

In [6]:
# transforms data in adata.X
adata.layers['log1pcpm'] = sc.pp.normalize_total(adata, target_sum=1e6, inplace=False)['X']

# transforms data in layers['lognorm'] inplace
sc.pp.log1p(adata, layer='log1pcpm')


In [7]:
import toml
dat = toml.load('../data/VISp_markers.toml')

In [8]:
# introduces "highly_variable" column to adata.var
sc.pp.highly_variable_genes(adata, 
                            layer='log1pcpm', 
                            flavor='cell_ranger',
                            n_top_genes=10000, 
                            inplace=True)

In [9]:
# Create new field with marker genes
adata.var['markers'] = np.isin(adata.var.index.values,dat['markers'])
adata

AnnData object with n_obs × n_vars = 13349 × 45768
    obs: 'seq_name', 'class', 'subclass', 'cluster', 'cell_types_98', 'cell_types_50', 'cell_types_25'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'markers'
    uns: 'log1p', 'hvg'
    layers: 'log1pcpm'

In [10]:
# This is a sparse matrix
adata.layers['log1pcpm']

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 120801360 stored elements and shape (13349, 45768)>

In [11]:
# For sparse matrix `M`, `M.todense()` to convert it to dense
adata.layers['log1pcpm'][:5,:5].toarray()

matrix([[0.        , 0.        , 3.84258223, 4.40539198, 0.        ],
        [0.        , 0.        , 4.16454502, 4.52873471, 0.42111822],
        [0.        , 0.        , 3.82509693, 3.56268307, 0.        ],
        [0.        , 0.        , 3.93543299, 0.        , 0.        ],
        [0.        , 0.        , 5.40677164, 4.62215864, 0.        ]])

**Parting notes:**

The anndata object created in this way has a few different fields that we will end up using with PERSIST
1. The raw counts are in `adata.X`
2. The normalized counts (log1p of CPM values) are in `adata.layers['log1pcpm']`
3. All metadata (cell type labels etc. for supervised mode in PERSIST) is in `adata.obs`
4. A coarse selection of genes is in `adata.var['highly_variable']`
5. Marker genes defined by Tasic et al. are indicated by `adata.var['markers']`

In [12]:
# adata_hvg is a view. We'll convert it to a new AnnData object and write it out. 
adata_hvg = ad.AnnData(X=adata.X,
                       obs=adata.obs, 
                       var=adata.var[['highly_variable']],
                       layers=adata.layers, uns=adata.uns)
adata_hvg.write('../data/VISp_filtered_cells.h5ad')